# III. Агрегирование данных


В этом разделе мы поговорим про агрегирование данных –

класификацию интервалов


## 3.2 По единицам АТД


In [ ]:
import geopandas as gpd

# Assuming theaters_gdf contains theater locations and okrug_gdf contains ATD polygons

# Perform spatial join to determine theaters in each ATD cell
theaters_in_atd = gpd.sjoin(theaters_gdf, okrug_gdf, op='within')

# Group by ATD cell and count theaters
theaters_count = theaters_in_atd.groupby('atd_id').size().reset_index(name='theater_count')

# Merge count information back to the original ATD GeoDataFrame
okrug_with_count = okrug_gdf.merge(theaters_count, on='atd_id', how='left')

# Save the updated GeoDataFrame to a new file
okrug_with_count.to_file('okrug_with_theater_count.shp')


## 3.1 По ячейкам регулярной сетки


### Квадратная


Check CRS of current data and decide do you need to reproject it or not?


- here is an example of reading the current crs and reprojecting geodataframe to another crs (you need to choose by yourself do you need it or not), but I'd rather reccommend to use UTM-zone, when you plot data with folium it is better to choose Web Mercator (EPSG:3857)


In [ ]:
print(data.crs.name)

#if you need to reproject you can use .to_crs method
data = data.to_crs("EPSG:4326")


Get the extent of the data


In [ ]:
total_bounds = data.total_bounds
minX, minY, maxX, maxY = total_bounds

Select cell-size


In [ ]:
- what are the units?

In [ ]:
square_size = 500

In [ ]:
grid_cells = []
x, y = (minX, minY)
geom_array = []

while y <= maxY:
        while x <= maxX:
            geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
            geom_array.append(geom)
            x += square_size
        x = minX
        y += square_size


fishnet = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs('EPSG:32637')
fishnet['id'] = fishnet.index

In [ ]:
fishnet.head()

Count points in polygon


In [ ]:
merged = gpd.sjoin(data, fishnet, how='left', predicate='within')

In [ ]:
merged

In [ ]:
dissolve = merged.dissolve(by="index_right", aggfunc="count")

In [ ]:
fishnet.loc[dissolve.index, 'n'] = dissolve.n.values

In [ ]:
fishnet

In [ ]:
fishnet.explore()

### Шестиугольная сетка (H3 library)


In [ ]:
import h3

# Создание шестиугольной сетки
resolution = 9  # разрешение H3
hexagons = [h3.geo_to_h3(lat, lon, resolution) for lat, lon in zip(theaters_gdf.geometry.y, theaters_gdf.geometry.x)]
theaters_gdf['hex_id'] = hexagons

In [ ]:
# Count theaters in each hexagonal cell
theaters_per_hex = theaters_gdf['hex_id'].value_counts().reset_index()
theaters_per_hex.columns = ['hex_id', 'theater_count']

## 3.3 Кластеризация
